In [ ]:
!pip install torch torchvision timm scikit-learn matplotlib seaborn pillow

## Training code 

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
import timm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import numpy as np
from PIL import Image

# ==========================
# 1. Configuration Settings
# ==========================

# Paths
DATA_DIR = '/content/images'  # Adjust path based on where the dataset is located in Colab
BEST_MODEL_PATH = 'best_efficientnet_b0.pth'
FINAL_MODEL_PATH = 'efficientnet_b0_final.pth'

# Hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 25
LEARNING_RATE = 1e-4
VALID_SPLIT = 0.2
RANDOM_SEED = 42

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==========================
# 2. Data Preparation
# ==========================

# Define transformations for training and validation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet means
                         std=[0.229, 0.224, 0.225]),   # ImageNet stds
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Load the dataset with ImageFolder
full_dataset = datasets.ImageFolder(root=DATA_DIR, transform=train_transforms)

# Calculate dataset sizes
total_size = len(full_dataset)
val_size = int(total_size * VALID_SPLIT)
train_size = total_size - val_size

# Split the dataset into training and validation
torch.manual_seed(RANDOM_SEED)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Update transforms for validation dataset
val_dataset.dataset.transform = val_transforms

# Handle class imbalance using WeightedRandomSampler
train_labels = [full_dataset.targets[idx] for idx in train_dataset.indices]
train_counts = Counter(train_labels)
print(f"Training class distribution: {train_counts}")

# Calculate weights for each class
class_weights = 1. / torch.tensor([train_counts[0], train_counts[1]], dtype=torch.float)
print(f"Class weights: {class_weights}")

# Assign weights to each sample
sample_weights = [class_weights[label] for label in train_labels]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(train_dataset), replacement=True)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# ==========================
# 3. Model Definition
# ==========================

# Initialize EfficientNet-B0
model = timm.create_model('efficientnet_b0', pretrained=True)

# Modify the classifier for binary classification
num_features = model.get_classifier().in_features
model.classifier = nn.Linear(num_features, 2)  # 2 classes: valid and invalid

# Move the model to the specified device
model = model.to(device)

# ==========================
# 4. Loss Function and Optimizer
# ==========================

# Define the loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# ==========================
# 5. Training and Validation Functions
# ==========================

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        # Predictions
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc

# ==========================
# 6. Training Loop
# ==========================

best_val_acc = 0.0

# Lists to store metrics
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 10)
    
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    # Step the scheduler
    scheduler.step()
    
    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print("Best model saved.\n")
    else:
        print()

# Save the final model after training
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"Final model saved as '{FINAL_MODEL_PATH}'")

# ==========================
# 7. Evaluation
# ==========================

# Load the best model
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model = model.to(device)
model.eval()

def get_predictions(model, dataloader, device):
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return all_preds, all_labels

# Get predictions and labels
preds, labels = get_predictions(model, val_loader, device)

# Calculate metrics
acc = accuracy_score(labels, preds)
precision = precision_score(labels, preds, average='binary', pos_label=1)
recall = recall_score(labels, preds, average='binary', pos_label=1)
f1 = f1_score(labels, preds, average='binary', pos_label=1)
cm = confusion_matrix(labels, preds)

print(f"Validation Accuracy: {acc:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Valid', 'Invalid'], 
            yticklabels=['Valid', 'Invalid'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


## inference code - 1

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import timm

# ==========================
# 1. Load the Model
# ==========================
def load_model(model_path, device):
    """
    Loads the trained model for inference.

    Args:
        model_path (str): Path to the saved model file.
        device (torch.device): Device to load the model onto.

    Returns:
        torch.nn.Module: The loaded model.
    """
    # Initialize EfficientNet-B0 model
    model = timm.create_model('efficientnet_b0', pretrained=False)

    # Modify the classifier for binary classification
    num_features = model.get_classifier().in_features
    model.classifier = torch.nn.Linear(num_features, 2)

    # Load the model weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

# Specify the path to the saved model
MODEL_PATH = 'efficientnet_b0_final.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = load_model(MODEL_PATH, device)
print("Model loaded successfully!")

# ==========================
# 2. Define Image Transformation
# ==========================
# The same transformations used during training and validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# ==========================
# 3. Prediction Function
# ==========================
def predict_image(image_path, model, transform, device):
    """
    Predicts whether an image is 'valid' or 'invalid'.

    Args:
        image_path (str): Path to the image file.
        model (torch.nn.Module): Trained model.
        transform (torchvision.transforms.Compose): Transformations to apply to the image.
        device (torch.device): Device to run the model on.

    Returns:
        str: 'valid' or 'invalid' based on prediction.
    """
    try:
        # Load and preprocess the image
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        # Perform inference
        with torch.no_grad():
            output = model(image)
            _, pred = torch.max(output, 1)

        # Return the class name
        return 'valid' if pred.item() == 0 else 'invalid'

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# ==========================
# 4. Test the Prediction
# ==========================
image_path = '/content/images/Valid_images/146_0091e3b5d5d3a595bea050b45a3a55512d5a6019cc5b9e50_7_P_DSC00723.JPG'  # Replace with the path to your image

# Get the prediction
prediction = predict_image(image_path, model, transform, device)

# Print the result
if prediction is not None:
    print(f"Prediction for the image '{image_path}': {prediction}")


## inference code -2 

In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import timm

# ==========================
# 1. Load the Model
# ==========================
def load_model(model_path, device):
    """
    Loads the trained model for inference.

    Args:
        model_path (str): Path to the saved model file.
        device (torch.device): Device to load the model onto.

    Returns:
        torch.nn.Module: The loaded model.
    """
    # Initialize EfficientNet-B0 model
    model = timm.create_model('efficientnet_b0', pretrained=False)

    # Modify the classifier for binary classification
    num_features = model.get_classifier().in_features
    model.classifier = torch.nn.Linear(num_features, 2)

    # Load the model weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

# Specify the path to the saved model
MODEL_PATH = 'efficientnet_b0_final.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = load_model(MODEL_PATH, device)
print("Model loaded successfully!")

# ==========================
# 2. Define Image Transformation
# ==========================
# The same transformations used during training and validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# ==========================
# 3. Predict Function for a Folder
# ==========================
def predict_folder(folder_path, model, transform, device):
    """
    Predicts the class (valid or invalid) for all images in a given folder.

    Args:
        folder_path (str): Path to the folder containing images.
        model (torch.nn.Module): Trained model.
        transform (torchvision.transforms.Compose): Transformations to apply to images.
        device (torch.device): Device to run the model on.

    Returns:
        List of tuples: Each tuple contains (image_name, prediction).
    """
    predictions = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        # Ensure it's a valid image file
        if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
            continue

        try:
            # Load and preprocess the image
            image = Image.open(image_path).convert('RGB')
            image = transform(image).unsqueeze(0).to(device)

            # Perform inference
            with torch.no_grad():
                output = model(image)
                _, pred = torch.max(output, 1)

            # Append the prediction
            class_name = 'valid' if pred.item() == 0 else 'invalid'
            predictions.append((image_name, class_name))

        except Exception as e:
            print(f"Error processing image {image_name}: {e}")

    return predictions

# ==========================
# 4. Run Inference on a Folder
# ==========================
folder_path = '/images/invalid_images'  # Replace with the folder path containing images

# Get predictions for all images in the folder
predictions = predict_folder(folder_path, model, transform, device)

# Print the predictions
print("\nPredictions:")
for image_name, class_name in predictions:
    print(f"Image: {image_name}, Predicted Class: {class_name}")

In [2]:
import os

def rename_images_in_folder(folder_path):
    """
    Renames all images in a folder sequentially as image1, image2, ..., imageN.

    Args:
        folder_path (str): Path to the folder containing the images.
    """
    # Supported image extensions
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')

    try:
        # Get a list of files in the folder
        files = os.listdir(folder_path)
        image_files = [f for f in files if f.lower().endswith(valid_extensions)]

        # Sort the files to maintain order
        image_files.sort()

        # Rename images
        for index, image_name in enumerate(image_files, start=1):
            old_path = os.path.join(folder_path, image_name)
            new_name = f"image{index}{os.path.splitext(image_name)[1]}"  # Keep original extension
            new_path = os.path.join(folder_path, new_name)

            os.rename(old_path, new_path)
            print(f"Renamed: {image_name} -> {new_name}")

        print("\nRenaming completed!")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
folder_path = "images\Valid_images"  # Get folder path from the user
rename_images_in_folder(folder_path)


<>:36: SyntaxWarning: invalid escape sequence '\V'
<>:36: SyntaxWarning: invalid escape sequence '\V'
C:\Users\sabar\AppData\Local\Temp\ipykernel_12024\2132123114.py:36: SyntaxWarning: invalid escape sequence '\V'
  folder_path = "images\Valid_images"  # Get folder path from the user


Renamed: 146_0091e3b5d5d3a595bea050b45a3a55512d5a6019cc5b9e50_7_P_DSC00723.JPG -> image1.JPG
Renamed: 146_0145ce4a676d43082edce7f5936021aa38507900a30b40e3_5_P_DSC01205.JPG -> image2.JPG
Renamed: 146_022c9120bd074453d2fbab2d3da8e0abb80ed2c0a59a2a8c_5_P_DSC01211.JPG -> image3.JPG
Renamed: 146_033ec4a25fd9737aff2212b5968ad3fe7f0b262f0f044e11_11_P_DSC01616.JPG -> image4.JPG
Renamed: 146_03ef5fa04b1f13dfa92c2246f2564e0b0660747b12f19796_4_P_P1264394.JPG -> image5.JPG
Renamed: 146_04c6e4caac1b1eab536a701152a83ae542e78e34fec6ded2_7_P_DSC01000.JPG -> image6.JPG
Renamed: 146_062cd1cfab98b4fd95415c65080b18ee3049a10544dc82f1_9_P_DSC01548.JPG -> image7.JPG
Renamed: 146_06a968d466c49823fd6c720ddafd94fcd51d3114a72b1b43_4_P_P1264582.JPG -> image8.JPG
Renamed: 146_078cebdb4ff3118d893481225868cf827d746e31d98c4acc_4_P_P1264426.JPG -> image9.JPG
Renamed: 146_07f9962646438e05844b4d09e16f29ccf37b337ebb30d7de_4_P_P1264024.JPG -> image10.JPG
Renamed: 146_082bb797aab99523a226be07b6b8db8f0cd24bf9dadc1834_5_P_P1